# lmr_analyzer Example
by Guilherme Fernandes Alves, Oct 2022 

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime

import boto3
from botocore import UNSIGNED
from botocore.config import Config

import lmr_analyzer as lmr

## Testing lmr_analyzer library

Testing package object definition

In [ ]:
example_pck = lmr.package(
    name="example_pck",
    dimensions=(25, 10, 7),  # width, height, length (cm)
    status="delivered",
    weight=1.5,  # kg
    price=9.90,  # $
)

example_pck.__dict__

Testing stops objects definition

In [ ]:
example_stp1 = lmr.stop(
    name="example_stp1",
    location=(0, 0),  # lat, lon
    location_type="pickup",
    time_window=(datetime(2020, 1, 1, 8, 0), datetime(2020, 1, 1, 8, 50)),
    packages=[example_pck],
    planned_service_time=0,
)
example_stp2 = lmr.stop(
    name="example_stp2",
    location=(0, 0),
    location_type="pickup",
    time_window=(datetime(2020, 1, 1, 9, 0), datetime(2020, 1, 1, 9, 10)),
    packages=[example_pck],
    planned_service_time=0,
)
example_stp1.__dict__

Testing route objects definition

In [ ]:
r1 = lmr.route(
    name="r1",
    stops=[example_stp1, example_stp2],
)
r1.set_actual_sequence(["example_stp1", "example_stp2"])
# r1.evaluate_route_scores()

r1.__dict__

In [ ]:
a1 = lmr.analysis(
    name="a1",
    routes=[r1],
)
a1.__dict__

## Serializing Amazon data

### Connecting to the Amazon database

Create a boto3 object with anonymized credentials

In [ ]:
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

List all the files available in the database.
It depends on your internet connection, it may take around 4 minutes.
Could be a good idea to save files locally after downloading. 

In [ ]:
!aws s3 ls --no-sign-request "s3://amazon-last-mile-challenges/almrrc2021/almrrc2021-data-training/model_build_inputs/"

Downloading the files

In [ ]:
!aws s3 cp --no-sign-request "s3://amazon-last-mile-challenges/almrrc2021/almrrc2021-data-training/model_build_inputs/" ../../data/last_mile_data/amazon-last-mile-challenges/  --recursive

### Run the serializing process

In [3]:
db = lmr.amzSerializer(
    root_directory="../../data/last_mile_data/amazon-last-mile-challenges/"
)

# Get some nicknames
packages_dict, routes_dict = db.packages_dict, db.routes_dict

route_data.json has been loaded in 11.28 seconds.
actual_sequences.json has been loaded in 1.36 seconds.
We are ready to proceed. All files have been loaded in 28.48 seconds.


In [4]:
db.print_info_by_city()

Number of routes in Los Angeles : 2876
Number of routes in Seattle     : 1079
Number of routes in Chicago     : 1002
Number of routes in Boston      : 929
Number of routes in Austin      : 214
Total number of routes:          6100

Percentage of routes in Los Angeles : 47.15%
Percentage of routes in Seattle     : 17.69%
Percentage of routes in Chicago     : 16.43%
Percentage of routes in Boston      : 15.23%
Percentage of routes in Austin      : 3.51%


## Create the distanceMatrix object

In [ ]:
dist_matrix = lmr.distanceMatrix()
dist_matrix.load_support_matrix_file(
    filename="../../data/driving_distances/los_angeles.csv"
)
dist_matrix.calculate_matrix_statistics()
dist_matrix.print_info()
# TODO: Add a __get__ method to the distanceMatrix class to return the matrix as a numpy array

## Analyze the data

In [ ]:
los_angeles = lmr.analysis(
    name="los_angeles",
    routes=list(routes_dict["Los Angeles"].values()),  # A list of route objects
)

los_angeles.calculate_euclidean_distances(planned=False, actual=True)
los_angeles.calculate_driving_distances(
    planned=False, actual_distance_matrix=dist_matrix.routes_matrix
)
los_angeles.calculate_circuity_factor(planned=False, actual=True)
los_angeles.calculate_packages_status()
los_angeles.calculate_centroids()
los_angeles.calculate_each_route_bbox()
los_angeles.find_overall_bbox()
df_sum_la = los_angeles.summarize_by_routes()

## Going beyond

In [ ]:
packages = 0
rejected = 0
failed_at = 0

for route in df_sum_la.values():
    packages += route.get("Number of packages", 0)
    rejected += route.get("Number of rejected packages", 0)
    failed_at += route.get("Number of failed attempted packages", 0)

# Calculate the percentage of rejected packages
rejected_p = 100 * rejected / packages
failed_p = 100 * failed_at / packages
print(f"Percentage of rejected packages: {rejected_p:.5}")
print(f"Percentage of failed attempted packages: {failed_p:.5}")

Export data to CSV

In [ ]:
los_angeles.summarize_by_routes()

In [ ]:
los_angeles.export_summary_by_routes(filename="export.csv")